<a href="https://colab.research.google.com/github/sugatoray/stackoverflow/blob/master/answers/Q_59258248.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solution to the following question
Stackoverflow question: [59258248](https://stackoverflow.com/questions/59258248/python-how-to-find-the-number-of-days-in-each-month-between-two-date-columns)

`python-how-to-find-the-number-of-days-in-each-month-between-two-date-columns`?

In [0]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import MonthEnd
from IPython.display import display

# Dummy Data
df = {'Id': ['1','2','3','4','5'],
      'Item': ['A','B','C','D','E'],
        'StartDate': ['2019-12-10', '2019-12-01', '2019-01-01', '2019-05-10', '2019-03-10'],
        'EndDate': ['2019-12-30' ,'2019-12-31','2019-03-30','2019-11-30','2019-06-10']
        }
df = pd.DataFrame(df,columns= ['Id', 'Item','StartDate','EndDate'])

# Function for Processing the DataFrame
def process_dataframe(df):
    """Returns the updated dataframe. """
    df.StartDate = pd.to_datetime(df.StartDate)
    df.EndDate = pd.to_datetime(df.EndDate)

    month_ends = pd.date_range(start='2019-01', freq='M', periods=12)    
    month_headers = month_ends.month_name().str.upper().str[:3].tolist()
    month_days = month_ends.day.to_numpy()
    month_nums = (np.arange(12) + 1)

    # Evaluate expressions to avoid multiple times evaluation
    start_date_month_num = df.StartDate.dt.month.to_numpy().reshape(-1,1)
    end_date_month_num = df.EndDate.dt.month.to_numpy().reshape(-1,1)

    #start_month_days = pd.to_datetime(df.StartDate, format="%Y%m") + MonthEnd(1) - df.StartDate
    # start_month_days.dt.days.to_numpy()
    # Number of days not in the end_month
    end_month_days_excluded = month_days[df.EndDate.dt.month.to_numpy() - 1] - df.EndDate.dt.day.to_numpy()

    # Determine the months that fall within the start and end dates (inclusive 
    # of start and end months) and then calculate the number of days in each 
    # month.

    # add all days for relevant months
    result = ((start_date_month_num <= month_nums) & \
            (end_date_month_num >= month_nums)).astype(int) \
            * month_days.reshape(1,-1) 
    # subtract number of days not in starting month
    result = result + \
            (-1) * (start_date_month_num == month_nums).astype(int) \
            * (df.StartDate.dt.day.to_numpy() - 1).reshape(-1,1) 
    # subtract number of days not in ending month            
    result = result + \
            (-1) * (end_date_month_num == month_nums).astype(int) \
            * end_month_days_excluded.reshape(-1,1) 

    return pd.merge(df, pd.DataFrame(result, columns = month_headers), left_index=True, right_index=True)

In [8]:
# Original DataFrame
display(df.head())

,Id,Item,StartDate,EndDate
0,1,A,2019-12-10,2019-12-30
1,2,B,2019-12-01,2019-12-31
2,3,C,2019-01-01,2019-03-30
3,4,D,2019-05-10,2019-11-30
4,5,E,2019-03-10,2019-06-10


In [9]:
# Updated DataFrame
df = process_dataframe(df)
display(df.head())

,Id,Item,StartDate,EndDate,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,1,A,2019-12-10,2019-12-30,0,0,0,0,0,0,0,0,0,0,0,21
1,2,B,2019-12-01,2019-12-31,0,0,0,0,0,0,0,0,0,0,0,31
2,3,C,2019-01-01,2019-03-30,31,28,30,0,0,0,0,0,0,0,0,0
3,4,D,2019-05-10,2019-11-30,0,0,0,0,22,30,31,31,30,31,30,0
4,5,E,2019-03-10,2019-06-10,0,0,22,30,31,10,0,0,0,0,0,0
